import file và thư viện

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scp
books = pd.read_csv('BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
users = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

Hiển thị bảng dữ liệu đánh giá sách

In [ ]:
print(ratings.shape)
#print(ratings)
print(list(ratings.columns))
#print(ratings.head(20))

Phân phối xếp hạng

In [ ]:
plt.rc("font", size=15)
ratings.bookRating.value_counts(sort=False).plot(kind='bar')
plt.title('Rating Distribution\n')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.savefig('Anh1.png', bbox_inches='tight')
plt.show()

Dữ liệu sách.

In [ ]:
print(books.shape)
print(list(books.columns))
books.head()

Dữ liệu người dùng

In [ ]:
print(users.shape)
print(list(users.columns))
users.head()

Biểu đồ phân bố độ tuổi

In [ ]:
users.Age.hist(bins=[0, 10, 20, 30, 40, 50, 100])
plt.title('Age Distribution\n')
plt.xlabel('Age')
plt.ylabel('Count')
plt.savefig('system2.png', bbox_inches='tight')
plt.show()

Thuật toán KNN:
Bắt đầu từ bộ dữ liệu ban đầu, chúng tôi sẽ chỉ xem xét những cuốn sách phổ biến. Để tìm ra cuốn sách nào phổ biến, chúng tôi kết hợp dữ liệu sách với dữ liệu xếp hạng

In [ ]:
combine_book_rating = pd.merge(ratings, books, on='ISBN')
columns = ['yearOfPublication', 'publisher', 'bookAuthor', 'imageUrlS', 'imageUrlM', 'imageUrlL']
combine_book_rating = combine_book_rating.drop(columns, axis=1)
combine_book_rating.head()

Sau đó, chúng tôi nhóm theo tên sách và tạo một cột mới cho tổng số xếp hạng.

In [ ]:
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['bookTitle'])

book_ratingCount = (combine_book_rating.
     groupby(by = ['bookTitle'])['bookRating'].
     count().
     reset_index().
     rename(columns = {'bookRating': 'totalRatingCount'})
     [['bookTitle', 'totalRatingCount']]
    )
book_ratingCount.head()

Chúng tôi kết hợp dữ liệu xếp hạng với tổng số dữ liệu xếp hạng, điều này cung cấp cho chúng tôi chính xác những gì chúng tôi cần để tìm ra cuốn sách nào phổ biến và lọc ra những cuốn sách ít được biết đến hơn.

In [ ]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'bookTitle', right_on = 'bookTitle', how = 'left')
rating_with_totalRatingCount.head()

Hãy xem số liệu thống kê về tổng số lượt xếp hạng:

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(book_ratingCount['totalRatingCount'].describe())

Cuốn sách trung bình đã được đánh giá chỉ một lần. Hãy nhìn vào đầu phân phối:

In [ ]:
print(book_ratingCount['totalRatingCount'].quantile(np.arange(.9, 1, .01)))

Lấy sách nhận được 50 xếp hạng trở lên

In [ ]:
popularity_threshold = 50
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_book.head()

Chỉ lọc người dùng ở Hoa Kỳ và Canada

In [ ]:
combined = rating_popular_book.merge(users, left_on = 'userID', right_on = 'userID', how = 'left')

us_canada_user_rating = combined[combined['Location'].str.contains("usa|canada")]
us_canada_user_rating=us_canada_user_rating.drop('Age', axis=1)
us_canada_user_rating.head()

Triển khai kNN

In [ ]:
us_canada_user_rating = us_canada_user_rating.drop_duplicates(['userID', 'bookTitle'])
us_canada_user_rating_pivot = us_canada_user_rating.pivot(index = 'bookTitle', columns = 'userID', values = 'bookRating').fillna(0)
us_canada_user_rating_matrix = scp.sparse.csr_matrix(us_canada_user_rating_pivot.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute',n_neighbors=7, n_jobs=-1)
model_knn.fit(us_canada_user_rating_matrix)

Lấy list BookTitle:

In [ ]:
us_canada_user_rating_pivot.info

In [ ]:
import os
import openpyxl

a = []
for i in range(0,2442):
    a.append(us_canada_user_rating_pivot.index[i])

#ghi ra file excel để đọc.
ListBookTitle = {"Ten Sach":[a]}
df_booktitle = pd.DataFrame(ListBookTitle)
#os.makedirs('E:\KPDLCODE_NguyenVanHoang_2019605266', exist_ok=True)  
df_booktitle.to_excel('E:\KPDLCODE_NguyenVanHoang_2019605266/data.xlsx') 
print("tao file thanh cong!")

In [ ]:
print('Nhap ten: ')
tensach = input()
for i in range(0,2442):
  if(tensach == a[i]):
    query_index = i
#query_index = np.random.choice(us_canada_user_rating_pivot.shape[0])

#print(query_index)
distances,indices=model_knn.kneighbors(us_canada_user_rating_pivot.iloc[query_index,:].values.reshape(1,-1),n_neighbors=10)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(us_canada_user_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, us_canada_user_rating_pivot.index[indices.flatten()[i]], distances.flatten()[i]))